# 中心軸の上下端点を指定するツール

## **Usage**

```main(file = "自動計測データ.xlsx", target=10)```

指定したエクセルファイルの「処理対象欄」に指定した数値が入っている個体だけが処理対象です。  
指定番号は実行時に 引数 target で指定できます。（デフォルトは10）

## 使い方

中心軸の上端と末端の２点をクリックして、ENTERで確定してください。 

座標と削除半径がエクセルファイルに記録されます。

- 「ESC」で終了します。(または「Q」または「０」）
- 「Enter」で確定次に進みます。
- 「R」でやり直しできます。ただし、上・下両方ともやり直しになります。
-  カーソルキー やアルファベットキーに削除半径の拡大縮小が割り当ててあります。


## キーバインド
|機能|キー|
|:---:|:---:|
|+1|→、d|
|+3|↑、w|
|-1|←、a|
|=3|↓、s|
|やり直し|r|
|確定|Enter（１回目は確認、２回目で本当に確定）|

私の MacBook では、ESC, Q, カーソルキー のキーコードが取得できませんでした。反応しない場合はプログラムを書き換えて別のキーを割り当ててください。

## おすすめ

上端の凸凹や末端が曲がりが激しい場所は形状近似に悪影響が出ますので、削除した方がよいでしょう。


# Excelファイルの仕様

省略、わかると思います。



In [18]:
import numpy as np
import cv2
import pandas as pd
import os
from rdlib2 import *
import matplotlib.pyplot as plt
# %matplotlib inline
from skimage.morphology import skeletonize
from skimage import morphology, color


# 重心の位置を求める   (rdlib2にある関数）
'''def getCoG(img):
    _lnum, _img, cnt, cog = cv2.connectedComponentsWithStats(img)
    areamax = np.argmax(cnt[1:,4])+1 # ０番を除く面積最大値のインデックス
    c_x,c_y = np.round(cog[areamax]) # 重心の位置を丸めて答える
    x,y,w,h = cnt[areamax,0:4] # 囲む矩形の x0,y0,w,h
    return c_x,c_y,x,y,w,h
'''
UNIT = 256

def main(file="計測指示＆記録.xlsx",target=10):
    global rdimg,sx,sy,rdcolor,rdcnt,uppercnt,lowercnt,c_x,c_y,x_r,y_r,radius,gcounter,tflag,bflag,dic,drd,df
    df = pd.read_excel(file)
    df.reset_index(inplace=True, drop=True) # エクセルファイルの編集でインデックスが欠落したり入れ替わっている場合があるとおかしくなるので振りなおしておく
  
    gcounter = 0
    
    def initcondition():
        global gcounter,radius,rdcolor,tflag,bflag,drd
        tflag,bflag = False,False # 上下の点が決まっているかどうかのフラグ
        radius = 10 # 円の半径
        drd = 0 # キー指定での半径加算分
        gcounter += 1
        rdcolor = rdcolor = cv2.cvtColor(rdimg, cv2.COLOR_GRAY2BGR)      
    
    for radish in range(len(df)):
        idata = df.iloc[radish]
        topdir = idata['topdir']  #  画像ファイルのパスのベース
        subdir = idata['subdir']  #  サブディレクトリ
        filename = idata['filename'] #  ファイル名
        check = idata['処理対象'] #  処理対象かどうかのフラグ　　test がTrueの時のみ意味がある
        if check !=target : #  check が 10 でない画像はスルーする
                continue

        path = os.path.join(topdir,subdir,filename)
        print("処理対象画像 {}\n".format(path))
        src= cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        org= cv2.imread(os.path.join('一軍',subdir,filename),1) 
        org = cv2.resize(org,(int(org.shape[1]/2),int(org.shape[0]/2)))
                
        rdimg = getstandardShape(src, unitSize=UNIT, thres=0.25, setrotation = 0, norotation = True)
        _img,contours,hierarchy = cv2.findContours(rdimg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        rdcnt = contours[np.argmax([len(c) for c in contours])] # 輪郭線情報　global 変数　
        
        c_x,c_y,x_r,y_r,width,height = getCoG(rdimg) # 重心の位置、バウンダリボックスの左上(x,y), 幅と高さ
        
        # スケルトンも候補に入れる
        rdimg1 = rdimg/255 # scikit-learn の細線化は１ビット画像でないといけない
        skimg = morphology.medial_axis(rdimg1)
        rdimg[skimg] = 128
        
        uppercnt = [[x,y] for [[x,y]] in rdcnt if y <= y_r + int(height/4)]
        lowercnt = [[x,y] for [[x,y]] in rdcnt if y >= y_r+int(0.5*height) ]
                                
        # スケルトン下部分の座標配列
        ys,xs = np.where(skimg)
        skpoints = [[x,y] for (x,y) in zip(xs,ys) if y >= y_r+int(0.5*height) ]
        # 各点について、最も近い輪郭点までの距離を求めておく
        radlist = []
        for p in skpoints:
            diff1 = [np.sqrt((p[0]-x)**2+(p[1]-y)**2) for [x,y] in lowercnt if x < p[0]]
            diff2 = [np.sqrt((p[0]-x)**2+(p[1]-y)**2) for [x,y] in lowercnt if x >= p[0]]
            if len(diff1) > 0 :
                if len(diff2) > 0:
                    lenfornearest = max(int(min(diff1)),int(min(diff2)))+3
                else:
                    lenfornearest = int(min(diff1))+3
            else:
                    lenfornearest = int(min(diff2))+3
                
            radlist.append(lenfornearest)
        dic = {}
        for [x,y],d in zip(skpoints,radlist):
            if d < 20:
                dic[(x,y)]=d
        
        skpoints = [key for key,d in zip(dic.keys(),dic.values()) if d < 20]
        
        lowercnt = lowercnt+skpoints
                            
        x0,y0,w0,h0 = cv2.boundingRect(rdcnt) # 外接矩形
        # print("x0 {} y0 {} width {} height {}".format(x0,y0,width,height))

        mu = cv2.moments(rdimg, False)
        gx,gy= int(mu["m10"]/mu["m00"]) , int(mu["m01"]/mu["m00"]) # 重心

        initcondition()       
        while True:
                wkimg = rdcolor.copy()
                cv2.namedWindow("PLOT")
                cv2.namedWindow(filename)
                cv2.setMouseCallback("PLOT", onMouse)
                cv2.imshow("PLOT", wkimg)
                cv2.imshow(filename,org)
                cv2.moveWindow(filename, 0, 500)
                key = cv2.waitKey(0)
                if key==ord('r') or key == ord('R'):
                    initcondition()
                elif key==2 or key==ord('1') or key==ord('a'): # LEFT 
                    radius = radius - 1  if radius > 2 else 0                    
                elif key==3 or key==ord('2') or key==ord('d'): # RIGHT
                    radius = radius + 1
                elif key==0 or key==ord('3') or key==ord('w'): # UP
                    radius = radius + 3
                elif key==1 or key==ord('4') or key==ord('s'): # DOWN
                    radius = radius - 3 if radius > 4 else 1
                elif key==ord('r'):
                    initcondition()
                elif key==13: # Enter で確定　次へ,
                    if not(tflag and bflag):
                        continue
                    font = cv2.FONT_HERSHEY_PLAIN
                    print("確定しますか? ENTER->確定  R -> やり直し")
                    cv2.putText(rdcolor,"OK?",(10,60),font,1,(0,255,0))
                    cv2.putText(rdcolor,"Ent->Record",(10,75),font,1,(0,255,0))
                    cv2.putText(rdcolor,"R->Reset",(10,90),font,1,(0,255,0))
                    key2 = cv2.waitKey(0)
                    if key2 == 13:
                        df.loc[radish,'TOPX'] = topdx
                        df.loc[radish,'TOPY'] = topdy
                        df.loc[radish,'TOPDR'] = topdr
                        df.loc[radish,'BTMX'] = btmdx
                        df.loc[radish,'BTMY'] = btmdy
                        df.loc[radish,'BTMDR'] = btmdr
                        df.loc[radish,'処理対象'] = 1
                        df.to_excel(file, index=True, header=True)
                        print("確定しました\n\n")
                        print("{} {} {}, {} {} {}".format(topdx,topdy,topdr,btmdx,btmdy,btmdr))
                        break
                    elif key2 == ord('r') or key2 == ord('R'):
                        initcondition()
                elif key==27: # ESC で終了
                    break
                [cx,cy],_d = nearestPos(sx,sy)
                drawcircle(wkimg)
        if key==27 or key==ord('q') or key == ord('0'): # ESC で終了
            cv2.destroyAllWindows()
            break
    df.to_excel(file, index=True, header=True)
    cv2.waitKey(1)
    
# (x,y)に最も近い輪郭上の点を答える
def nearestPos(x,y):
    # global  rdcnt,uppercnt,lowercnt,c_x,c_y,x_r,y_r
    cnt = uppercnt if y <= c_y else lowercnt
    diff = [[x-cx,y-cy] for [cx,cy] in cnt] # 輪郭点と(x,y)を結ぶベクトル
    distance = [np.sqrt(dx*dx+dy*dy) for dx,dy in diff] # ベクトルの長さ = 距離
    mi = np.argmin(distance)
    return cnt[mi],distance[mi]

def drawcircle(wkimg):
    global sx,sy,radius,dic,drd
    [cx,cy],d_ = nearestPos(sx,sy)
    if (cx,cy) in dic:
        if radius < dic[(cx,cy)]:
            drd = dic[(cx,cy)]-radius
    else:
        drd = 0
    cv2.circle(wkimg,(cx,cy),5,(0,0,255),-3)
    cv2.circle(wkimg,(cx,cy),radius+drd,(255,0,255),2)
    cv2.imshow("PLOT", wkimg)
    
def drawcursor(wkimg):
    [h,w] = wkimg.shape[:2]
    cv2.line(wkimg,(sx,0),(sx,h),(255,0,0),1)
    cv2.line(wkimg,(0,sy),(w,sy),(255,0,0),1)       

# マウスのコールバック関数　マウスイベントに対する応答
def onMouse(event, x, y, flags,params):    
    global rdimg,sx,sy,rdcolor,radius,sx,sy,tflag,bflag,topdx,topdy,topdr,btmdx,btmdy,btmdr
    sx,sy = x,y
    wkimg = rdcolor.copy()
    # クリックされた時
    if event == cv2.EVENT_LBUTTONUP:
        sx, sy = x, y # マウスの位置をグローバル変数に代入して処理ルーチンに伝える
        [cx,cy] ,distance= nearestPos(sx,sy)
        print("({},{}) - ({},{}) distance {}".format(x,y,cx,cy,distance))
        if y <= c_y and tflag == False: # 上端確定
            topdx,topdy,topdr = cx,cy,radius
            tflag = True
        elif y > c_y and bflag == False:
            btmdx,btmdy,btmdr = cx,cy,radius+drd
            bflag = True
        else:
            return
        cv2.circle(rdcolor,(cx,cy),radius+drd,(0,0,255),-1)
        cv2.circle(rdcolor,(cx,cy),3,(255,255,255),-1)
    # マウスが移動ている間は十字カーソルを表示
    if event == cv2.EVENT_MOUSEMOVE:
        wkimg = rdcolor.copy()
        drawcursor(wkimg)
        drawcircle(wkimg)

In [20]:
main(file="計測指示＆記録.xlsx",target=3)
cv2.destroyAllWindows()
cv2.waitKey(1)

処理対象画像 シルエット/17Cylindric/17daruma6o09_l.jpg

(202,303) - (201,303) distance 1.0
(182,64) - (182,65) distance 1.0
確定しますか? ENTER->確定  R -> やり直し
確定しました


182 65 16, 201 303 10
処理対象画像 シルエット/17Cylindric/17kohaku1o02_l.jpg

(203,308) - (203,308) distance 0.0
(202,63) - (202,64) distance 1.0
確定しますか? ENTER->確定  R -> やり直し
確定しました


202 64 19, 203 308 10
処理対象画像 シルエット/17Cylindric/17kohaku1o09_l.jpg

(207,303) - (207,303) distance 0.0
(194,62) - (194,64) distance 2.0
確定しますか? ENTER->確定  R -> やり直し
確定しました


194 64 16, 207 303 10
処理対象画像 シルエット/17Cylindric/17kohaku2o10_l.jpg

(194,64) - (193,65) distance 1.4142135623730951
(186,318) - (186,317) distance 1.0
確定しますか? ENTER->確定  R -> やり直し
確定しました


193 65 16, 186 317 10
処理対象画像 シルエット/17Cylindric/17makoto1o01_l.jpg

(195,312) - (195,311) distance 1.0
(195,60) - (193,64) distance 4.47213595499958
確定しますか? ENTER->確定  R -> やり直し
確定しました


193 64 16, 195 311 10
処理対象画像 シルエット/17Cylindric/17makoto1o05_l.jpg

(188,299) - (188,299) distance 0.0
(193,74) - (193,65) distanc

-1

5.3